# Milestone Project 2: SkimLit

The purpose of this notebook is to build an NLP model to make reading medical abstracts easier.

The paper we're replicating is avaliable here: [PubMed 200k RCT](https://arxiv.org/abs/1710.06071)

Reading through the paper above, we see that the model architecture that they use to achieve their best results is availbale here: [Neural Networks for Joint Sentence Classification](https://arxiv.org/abs/1612.05251)

## Get data

Since we are replicating the model from the above paper, let's use their data.

[Dataset](https://github.com/Franck-Dernoncourt/pubmed-rct)

In [1]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (15/15), done.
PubMed_200k_RCT
PubMed_200k_RCT_numbers_replaced_with_at_sign
PubMed_20k_RCT
PubMed_20k_RCT_numbers_replaced_with_at_sign
README.md


In [2]:
# Check what files are in the PubMed20k dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

dev.txt   test.txt  train.txt


In [15]:
# Start our experiments using the 20k dataset with numbers replaced by '@' sign
data_dir = '/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Milestone Projects/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/'

In [16]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Milestone Projects/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 '/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Milestone Projects/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt',
 '/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Milestone Projects/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt']

## Preprocess data

Now we've got some text data, it's time to get one with it.

In [17]:
# Create a function to read the lines of the document
def get_lines(filename):
    '''
    Reads filename and returns the lines of text as a list.

    Args:
        filename (str): Path of filename
    
    Returns:
        A list of string with one string per line from the target filename
    '''

    with open(filename, 'r') as f:
        return f.readlines()

In [19]:
# Let's read in the training lines
train_lines = get_lines(filenames[1])
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

Let's think about how we want our data to look

Representation of the data:

```
[{'line_number': 0,
    'target': 'BACKGROUND',
    'text': 'xyz',
    'total_lines': 11}]
```

In [20]:
def preprocess_text_with_line_numbers(filename):
    '''
    Returns a list of dictionary of abstract line data.

    Takes in filename, reads it contents and sorts through each line, extracting things like the target label, the text of the sentence, number of snetences and what sentence number line is.
    '''
    input_lines = get_lines(filename)
    abstract_lines = ""
    abstract_samples = []

    for line in input_lines: # if the line is the id number
        if line.startswith("###"):
            abstract_id = line
            abstract_lines = ""
        elif line.isspace(): # if it is the new line
            abstract_line_split = abstract_lines.splitlines()

            for line_number, abstract_line in enumerate(abstract_line_split):
                line_data = {}
                target_text_split = abstract_line.split('\t')
                line_data['target'] = target_text_split[0]
                line_data['text'] = target_text_split[1].lower()
                line_data['line_number'] = line_number
                line_data['total_lines'] = len(abstract_line_split) - 1
                abstract_samples.append(line_data)

        else: # if it is the text
            abstract_lines += line

    return abstract_samples

In [24]:
train_samples = preprocess_text_with_line_numbers(filenames[1])
val_samples = preprocess_text_with_line_numbers(filenames[0])
test_samples = preprocess_text_with_line_numbers(filenames[2])

len(train_samples), len(val_samples), len(test_samples)

(180040, 30212, 30135)

In [27]:
# Check the first abstract
train_samples[:14]

[{'target': 'OBJECTIVE',
  'text': 'to investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( oa ) .',
  'line_number': 0,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'a total of @ patients with primary knee oa were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .',
  'line_number': 1,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
  'line_number': 2,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'pain was assessed using the visual analog pain scale ( @-@ mm ) .',
  'line_number': 3,
  'total_lines': 11},
 {'target': 'METHODS',
  'text': 'secondary outcome measures included the western ontari